<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_Pytorch/blob/main/Model/GPT_2/GPT2_GEN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sentencepiece : https://github.com/google/sentencepiece/blob/master/python/README.md

### NSMC

In [72]:
import urllib

In [73]:
urllib.request.urlretrieve('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt','train.txt')

('train.txt', <http.client.HTTPMessage at 0x79a87b7f9b40>)

In [69]:
cd ~

/root


In [67]:
temp=[]

In [74]:
with open('train.txt','r') as f:
  f.readline()
  for sent in f.readlines():
    data = sent.split('\t')
    temp.append(data[1])

In [ ]:
len(temp)

150000

In [ ]:
cd drive/MyDrive/Pytorch\ NLP/GPT-2/data_in/gpt2_ckpt

/content/drive/MyDrive/Pytorch NLP/GPT-2/data_in/gpt2_ckpt


In [ ]:
with open('corpus.txt','w') as f:
  for sent in temp:
    f.write(f'\n {sent}')

### Sentencepiece

https://keep-steady.tistory.com/7

In [6]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.3 MB/s eta 0:00:00


In [7]:
import sentencepiece as spm

In [ ]:
spm.SentencePieceTrainer.train(input='corpus.txt',model_prefix='kor_tokenizer',vocab_size=50257, model_type='bpe',max_sentence_length=9999999,pad_id=0,unk_id=1,bos_id=2,eos_id=3)

### Tokenizer

In [11]:
sp=spm.SentencePieceProcessor()
sp.Load('kor_tokenizer.model')

True

In [66]:
sp.EncodeAsPieces('안녕하세요 저는 사람이에요.')

['▁안녕하세요', '▁저는', '▁사람이', '에요', '.']

In [ ]:
sp.encode('안녕하세요 저는 사람이에요.')

[20363, 1619, 1047, 802, 48546]

In [ ]:
sp.encode('그래')

[312]

In [ ]:
sp.bos_id()

2

In [ ]:
vocab = [sp.id_to_piece(id) for id in range(sp.get_piece_size())]

In [ ]:
print('0 :{}, 1 :{} , 2 :{}, 3 :{} '.format(vocab[0],vocab[1],vocab[2],vocab[3]))

0 :<pad>, 1 :<unk> , 2 :<s>, 3 :</s> 


## Input for fine-tuning

### filtering

In [75]:
import re

In [76]:
temp[5]

'막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.'

In [77]:
filtered_sent = [ ' '.join((re.sub('[^가-힣0-9 ]',' ',sent)).split()) for sent in temp]

In [80]:
filtered_sent[11]

'볼때마다 눈물나서 죽겠다90년대의 향수자극 허진호는 감성절제멜로의 달인이다'

### padding and truncation

In [ ]:
import numpy as np

In [ ]:
tokenized_len=[len(sp.encode(sent)) for sent in filtered_sent ]

In [ ]:
print('3사분위 길이 :{}'.format(np.percentile(tokenized_len,99)))

3사분위 길이 :47.0


In [81]:
MAX_LEN=47

In [82]:
import torch

In [83]:
def preprocess_input_output(input,max_len):
  # input : (samples, sent_len)

  train_input=[]
  train_output=[]

  # append <s>, </s>, <pad>
  for sent in input:
    if len(sp.encode(sent))<max_len:
      pad_len = max_len-len(sp.encode(sent))-1
      train_input.append([sp.bos_id()]+sp.encode(sent)+[sp.pad_id()]*pad_len)
      train_output.append(sp.encode(sent)+[sp.eos_id()]+[sp.pad_id()]*pad_len)
    else: # truncation
      train_input.append([sp.bos_id()]+sp.encode(sent)[:max_len-1])
      train_output.append(sp.encode(sent)[:max_len-1]+[sp.eos_id()])

  return torch.LongTensor(train_input), torch.LongTensor(train_output) # (samples, max_len)

In [84]:
train_input, train_output = preprocess_input_output(filtered_sent,MAX_LEN)

In [ ]:
train_input[0]

tensor([    2,     8,  1080,    55, 17382,  2106,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])

In [ ]:
train_output[0]

tensor([    8,  1080,    55, 17382,  2106,     3,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0])

In [85]:
from torch.utils.data import Dataset,DataLoader

In [86]:
class CustomDataset(Dataset):
  def __init__(self,x,y):
    self.x=x
    self.y=y

  def __getitem__(self,index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.x.size()[0]

In [87]:
train_dataset = CustomDataset(train_input,train_output)

In [90]:
train_iter = DataLoader(train_dataset,batch_size=64)

In [89]:
next(iter(train_iter))

[tensor([[    2,     8,  1080,  ...,     0,     0,     0],
         [    2,  1609, 17387,  ...,     0,     0,     0],
         [    2,    25, 48579,  ...,     0,     0,     0],
         ...,
         [    2,   194, 48700,  ...,     0,     0,     0],
         [    2, 39386,  1661,  ...,     0,     0,     0],
         [    2,  4928,  3547,  ...,     0,     0,     0]]),
 tensor([[    8,  1080,    55,  ...,     0,     0,     0],
         [ 1609, 17387, 23046,  ...,     0,     0,     0],
         [   25, 48579, 50166,  ...,     0,     0,     0],
         ...,
         [  194, 48700, 48547,  ...,     0,     0,     0],
         [39386,  1661, 38441,  ...,     0,     0,     0],
         [ 4928,  3547,   610,  ...,     0,     0,     0]])]

### config & model_weight

In [ ]:
urllib.request.urlretrieve('https://huggingface.co/gpt2/resolve/main/config.json','config.json')

('config.json', <http.client.HTTPMessage at 0x7fdd07c1ada0>)

In [ ]:
urllib.request.urlretrieve('https://huggingface.co/gpt2/resolve/main/model.safetensors','model.safetensors')

('model.safetensors', <http.client.HTTPMessage at 0x7fdd07c19ed0>)

In [94]:
cd drive/MyDrive/Pytorch\ NLP/GPT-2/data_in

/content/drive/MyDrive/Pytorch NLP/GPT-2/data_in


In [3]:
pip install wget

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=4ab74ff4e824669769cb28e4fd058e4a74225cd1b74e2f8689bd7d1bba7de6a8
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [4]:
import wget
import zipfile

wget.download('https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/releases/download/v1.0/gpt_ckpt.zip')

with zipfile.ZipFile('gpt_ckpt.zip') as z:
    z.extractall()

## Top k & Top p

### torch.multinomial

In [12]:
import torch
import torch.nn as nn

In [13]:
def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
  # logits = (vocab_size, )

  if top_k>0:
    top_k=min(top_k,logits.size()[0])
    indices_to_remove=logits<torch.topk(logits,top_k).values[-1] # smaller than the smallest value among the top_k values
    logits[indices_to_remove]=filter_value

  if top_p>0.0:
    sorted_logits=torch.sort(logits,descending=True)
    sorted_logits_index=torch.argsort(logits,descending=True)
    prob_cumsum = torch.cumsum(nn.Softmax()(sorted_logits))

    sorted_indices_to_remove=prob_cumsum>top_p
    sorted_indices_to_remove=torch.cat([torch.LongTensor([False]),sorted_indices_to_remove[:-1]]) # prevent if first is True
    indices_to_remove = sorted_logits_index[sorted_indices_to_remove]

    logits[indices_to_remove]=filter_value

  return logits   # logits = (vocab_size, )


## Greedy & Generate_sentence Function

In [52]:
def generate_sent(seed_word,model,max_len=100,greedy=False,top_k=0,top_p=0.0):
  sent= seed_word
  toked = sp.encode(sent)

  for _ in range(max_len):
    input_ids = torch.LongTensor([sp.bos_id()]+toked)[None,:] # input_ids = (1, cumulated_seq_len)
    outputs = model(input_ids)[0,-1,:] # outputs : (vocab_size, )

    if greedy:
      gen = sp.id_to_piece(outputs.argmax().tolist()) # outputs.argmax().tolist() -> int
    else:
      output_logits = top_k_top_p_filtering(outputs,top_k,top_p) # logits = (vocab_size, )
      gen = torch.multinomial(nn.Softmax(-1)(output_logits),num_samples=1,replacement=True).tolist()[0]
      gen = sp.id_to_piece(gen)
    if gen == '</s>':
      break

    sent+=gen.replace('▁',' ')
    toked=sp.encode(sent)

  return sent

## Huggingface Transformers

In [16]:
cd ..

/content/drive/MyDrive/Pytorch NLP/GPT-2/data_in


In [15]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.8 MB/s eta 0:00:00


In [17]:
from transformers import GPT2LMHeadModel

## Model Pre-training

In [111]:
class GPT2Gen(nn.Module):
  def __init__(self,dir_path):
    super(GPT2Gen,self).__init__()
    self.gpt2 = GPT2LMHeadModel.from_pretrained(dir_path,ignore_mismatched_sizes=True)

  def forward(self,x):
    outputs=self.gpt2(x)[0]
    return torch.transpose(outputs,2,1) # (batch_size=1, vocab_size, max_len)

In [118]:
# hyperparameter
MAX_LEN=47
BATCH_SIZE=32
EPOCHS=10

device='cuda' if torch.cuda.is_available() else 'cpu'

In [112]:
model = GPT2Gen('./data_in/gpt2_ckpt').to(device)
criterion = torch.nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(),lr=3e-5)

In [108]:
for name,param in model.named_parameters():
  if not name=='gpt2.transformer.wte.weight':
    param.requires_grad=False

In [109]:
for name,param in model.named_parameters():
  print(f'{name}, {param.requires_grad}')

gpt2.transformer.wte.weight, True
gpt2.transformer.wpe.weight, False
gpt2.transformer.h.0.ln_1.weight, False
gpt2.transformer.h.0.ln_1.bias, False
gpt2.transformer.h.0.attn.c_attn.weight, False
gpt2.transformer.h.0.attn.c_attn.bias, False
gpt2.transformer.h.0.attn.c_proj.weight, False
gpt2.transformer.h.0.attn.c_proj.bias, False
gpt2.transformer.h.0.ln_2.weight, False
gpt2.transformer.h.0.ln_2.bias, False
gpt2.transformer.h.0.mlp.c_fc.weight, False
gpt2.transformer.h.0.mlp.c_fc.bias, False
gpt2.transformer.h.0.mlp.c_proj.weight, False
gpt2.transformer.h.0.mlp.c_proj.bias, False
gpt2.transformer.h.1.ln_1.weight, False
gpt2.transformer.h.1.ln_1.bias, False
gpt2.transformer.h.1.attn.c_attn.weight, False
gpt2.transformer.h.1.attn.c_attn.bias, False
gpt2.transformer.h.1.attn.c_proj.weight, False
gpt2.transformer.h.1.attn.c_proj.bias, False
gpt2.transformer.h.1.ln_2.weight, False
gpt2.transformer.h.1.ln_2.bias, False
gpt2.transformer.h.1.mlp.c_fc.weight, False
gpt2.transformer.h.1.mlp.c_fc.b

In [99]:
from tqdm import tqdm

In [ ]:
a=torch.Tensor([[1,1,1,0,0],[1,1,0,0,0]])
mask=~(a==0)

b=torch.Tensor([[3,2,5,0,0],[1,5,3,1,0]])
(b*mask).sum()

tensor(16.)

In [100]:
def compute_loss(outputs,y,criterion):
  loss = criterion(outputs,y) # (batch_size, max_len)
  mask = ~(y==0)

  return (loss*mask).sum()

In [104]:
def train(model,optimizer,criterion,train_iter):
  model.train()
  losses=[]

  total_loss = 0
  for batch in tqdm(train_iter):
    x=batch[0].to(device)
    y=batch[1].to(device)

    outputs = model(x) # outputs : (batch_size, vocab_size, max_len)
    loss = compute_loss(outputs,y,criterion) # (batch_size, max_len )
    total_loss+=loss
    losses.append(loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  return total_loss/len(train_iter), losses

In [102]:
import os

In [116]:
best_loss = None

for e in range(EPOCHS):
  avg_loss, losses=train(model,optimizer,criterion,train_iter)
  print('avg_loss : {}'.format(avg_loss))

  if not best_loss or avg_loss<best_loss:
    os.makedirs('./data_out/best_model')
    torch.save(model.state_dict(),'./data_out/best_model/best_weight.pt')
    best_loss=avg_loss

  0%|          | 0/2344 [00:00<?, ?it/s]


OutOfMemoryError: ignored

In [ ]:
losses

In [ ]:
model.state_dict().keys()

odict_keys(['gpt2.transformer.wte.weight', 'gpt2.transformer.wpe.weight', 'gpt2.transformer.h.0.ln_1.weight', 'gpt2.transformer.h.0.ln_1.bias', 'gpt2.transformer.h.0.attn.c_attn.weight', 'gpt2.transformer.h.0.attn.c_attn.bias', 'gpt2.transformer.h.0.attn.c_proj.weight', 'gpt2.transformer.h.0.attn.c_proj.bias', 'gpt2.transformer.h.0.ln_2.weight', 'gpt2.transformer.h.0.ln_2.bias', 'gpt2.transformer.h.0.mlp.c_fc.weight', 'gpt2.transformer.h.0.mlp.c_fc.bias', 'gpt2.transformer.h.0.mlp.c_proj.weight', 'gpt2.transformer.h.0.mlp.c_proj.bias', 'gpt2.transformer.h.1.ln_1.weight', 'gpt2.transformer.h.1.ln_1.bias', 'gpt2.transformer.h.1.attn.c_attn.weight', 'gpt2.transformer.h.1.attn.c_attn.bias', 'gpt2.transformer.h.1.attn.c_proj.weight', 'gpt2.transformer.h.1.attn.c_proj.bias', 'gpt2.transformer.h.1.ln_2.weight', 'gpt2.transformer.h.1.ln_2.bias', 'gpt2.transformer.h.1.mlp.c_fc.weight', 'gpt2.transformer.h.1.mlp.c_fc.bias', 'gpt2.transformer.h.1.mlp.c_proj.weight', 'gpt2.transformer.h.1.mlp.c_pr

In [113]:
model.train()

GPT2Gen(
  (gpt2): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  )
)

## Text Generative Model

In [24]:
cd ..

/content/drive/MyDrive/Pytorch NLP/GPT-2


In [20]:
class GPT2Gen(nn.Module):
  def __init__(self,dir_path):
    super(GPT2Gen,self).__init__()
    self.gpt2 = GPT2LMHeadModel.from_pretrained(dir_path,ignore_mismatched_sizes=True)

  def forward(self,x):
    return self.gpt2(x)[0] # (batch_size=1, cumulated_seq_len, vocab_size =500257)

In [25]:
gpt2_model = GPT2Gen('./data_in/gpt2_ckpt')

Generate_sentences

In [ ]:
model.load_state_dict(torch.load('data_out/best_model/best_weight.pt'))

<All keys matched successfully>

In [ ]:
model.to('cpu')

GPT2Gen(
  (gpt2): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 768)
      (wpe): Embedding(1024, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-11): 12 x GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (lm_head): Linear(in_features=768, out_features=50257, bias=False)
  )
)

In [56]:
generate_sent('안녕',gpt2_model,max_len=10,top_k=10)

'안녕 차 그래도 보겠다 이수 그근한 포스터만드라 그 커'

In [ ]:
""